In [1]:
import os
from google.cloud import bigquery
import pandas as pd
import hashlib
import random

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'credentials.json'

client = bigquery.Client()

query = """
    WITH eventos_agg AS (
      SELECT
        user_pseudo_id,
        COUNTIF(event_name = "purchase") AS n_compras,
        COUNTIF(event_name = "add_to_cart") AS n_adds,
        COUNTIF(event_name = "view_item") AS n_views,
        SUM(IFNULL(revenue, 0)) AS total_receita,
        COUNT(DISTINCT event_date) AS dias_ativos,
        APPROX_TOP_COUNT(item_category, 1)[OFFSET(0)].value AS top_category
      FROM (
        SELECT
          user_pseudo_id,
          event_name,
          event_date,
          (SELECT value.double_value FROM UNNEST(event_params) WHERE key = "value") AS revenue,
          (SELECT value.string_value FROM UNNEST(event_params) WHERE key = "item_category") AS item_category
        FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
        WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
          AND event_name IN ("purchase", "view_item", "add_to_cart")
      )
      GROUP BY user_pseudo_id
    )
    SELECT * FROM eventos_agg
    """

df = client.query(query).to_dataframe()

c:\Users\pedro\Projetos_Python\segmentacao_remarketing\segmentacao_remarketing\seg_mkt\lib\site-packages\google\cloud\bigquery\table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61284 entries, 0 to 61283
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_pseudo_id  61284 non-null  object 
 1   n_compras       61284 non-null  Int64  
 2   n_adds          61284 non-null  Int64  
 3   n_views         61284 non-null  Int64  
 4   total_receita   61284 non-null  float64
 5   dias_ativos     61284 non-null  Int64  
 6   top_category    0 non-null      object 
dtypes: Int64(4), float64(1), object(2)
memory usage: 3.5+ MB


In [3]:
df.head()

,user_pseudo_id,n_compras,n_adds,n_views,total_receita,dias_ativos,top_category
0,2291182.3518179459,2,3,28,20.80,4,None
1,55916158.1192891856,1,9,29,0.00,1,None
2,5572774684.5698209837,0,3,28,0.00,8,None
3,3680421.4213771360,4,26,157,92.64,5,None
4,5318101.4795757171,1,10,18,0.00,2,None
